# Avaliação 2

### Questão 1
Utilizando a base Titanic disponível no kaggle (https://www.kaggle.com/c/titanic/data) elabore uma solução utilizando dois algoritmos de aprendizagem de máquina do seu conhecimento para classificar se o passageiro tem ou não chance de sobreviver. Os resultados dessa questão deverão ser descritos detalhadamente no relatório.
* Lembre de fazer todo o pré-processamento, explicando suas decisões;
* Avalie os resultados usando diferentes métricas.
* Teste 3 variações de parâmetros dos métodos;

#### Pré-processamento

É dado início ao pré-processamento, importando as bibliotecas necessárias e instanciando os databases forneceidos através do pandas.

É importante entender que:
* __gs__ equivale à instância do database que contém as saídas (coluna *Survived*) corretas para os testes
* __tr__ equivale à instância do database que contém os treinos
* __ts__ equivale à instância do database que contém os testes

Vale lembrar que o database __gs__ refere-se ao __teste__, e que o __teste__ não pertence ao treino. Portanto será interessante unir os dois databases futuramente.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn import cluster, neighbors, svm, metrics, preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

gs = pd.read_csv("data/gender_submission.csv",sep=",")
tr = pd.read_csv("data/train.csv",sep=",")
ts = pd.read_csv("data/test.csv",sep=",")

#### Avaliando a quantidade de itens em cada database

Para entender a quantidade de itens com o qual se está trabalhando, é feita uma leitura através do comando __*len()*__.

In [ ]:
print("Tamanho do database de treino:", len(tr))
print("Tamanho do database de testes:", len(ts))
print("Tamanho do database que contém as saídas corretas do teste:", len(gs))

#### Unindo todos os databases

Para passarem pelo pré-processamento, é interessante que todos os dados estejam juntos. Desta forma o __gs__ será unido ao __ts__, que será, por sua vez, unido ao __tr__.

❗️ __Utilizaremos o termo *dfTotal* para se referir ao DataFrame completo, que inclui o teste e o treino.__ ❗️

In [ ]:
dfTotal = pd.concat([tr,pd.concat([gs, ts.drop(columns=['PassengerId'])], axis=1)])

#### Analisando quantos itens nulos há no database

É importante conhecer quantos itens nulos existem no database para entender qual deve ser a medida adequada a se tomar mediante a quantidade de itens nulos.

In [ ]:
print("-->Nulidade:")
print(dfTotal.isnull().sum())

⚠️ __Os itens nulos de cabine representam, no treino, mais de 77% do total de informações. Excluir todas essas informações pode representar uma alta perda ao treinamento do IA, ao passo que completar todo esse conteúdo com informações aleatórias ou fixas também podem trazer algum nível de imprecisão aos testes. Desta forma, optou-se por excluir essa coluna.__ ⚠️

#### Excluindo a coluna de Cabine

Como dito acima, a coluna *Cabin* apresenta muitos valores nulos. Desta forma passa a ser interessante excluir essa coluna.

In [ ]:
dfTotal = dfTotal.drop(columns=['Cabin'])

#### Analisando quantos itens nulos há no database

Ainda analisando a quantidade de elementos nulos, será feita a análise novamente agora sem a coluna de cabine, que previamente apresentou várias informações faltantes.

In [ ]:
print("-->Nulidade:")
print(dfTotal.isnull().sum())

❗️ __Considerando que *Fare* (Tarifa) e *Embarked* apresentam uma quantidade ínfima de valores nulos, considerou-se interessante, na coluna Fare, considerar a mediana no lugar do valor nulo e, no Embarked, o uso da moda dos valores.__ ❗️

#### Substituindo valores de Fare, Age e Embarked

Será considerada a moda dos valores de __Embarked__ para substituir os valores nulos e a mediana de __Fare__ e __Age__ para substituir os valores faltantes.

Note que, por mais que haja uma quantidade mais expressiva de valores nulos para *Age*, isso representa apenas 20%, da quantidade total. Sendo assim é entendeu-se como viável substituir os valores de idade pela mediana de todas as idades.

In [ ]:
dfTotal["Fare"].fillna(dfTotal["Fare"].median(), inplace=True)
print(dfTotal.isnull().sum())

In [ ]:
dfTotal["Embarked"].fillna(dfTotal["Embarked"].mode()[0],inplace=True)
print(dfTotal.isnull().sum())

In [ ]:
dfTotal["Age"].fillna(dfTotal["Age"].median(),inplace=True)
print(dfTotal.isnull().sum())

#### Removendo colunas

Algumas colunas não auxiliam, pelo contrário, atrapalham o funcionamento do IA por apresentarem valores únicos para cada entidade. Desta forma, devem ser removidas as colunas:
* Name
* Ticket
* PassengerId


In [ ]:
dfTotal = dfTotal.drop(columns=['Name','Ticket','PassengerId'])

#### Linhas duplicadas

Com a remoção das colunas, algumas linhas possuem valores exatamente iguais, não agregando em nada ao IA. Sendo assim, primeiramente será feita a análise de quantas linhas estão duplicadas:

In [ ]:
print("Total de duplicações no database:",dfTotal.duplicated().sum())

#### Remoção de linhas duplicadas

Existem 209 linhas duplicadas. Agora é interessante apagar essas duplicações que não agregam em nada ao IA.

In [ ]:
dfTotal = dfTotal.drop_duplicates()
print("Conferindo o total de duplicações após apagar as linhas duplicadas:",dfTotal.duplicated().sum())

#### Substituindo valores

Como um dos métodos a ser utilizado na análise será o KNN, então é necessário que todas as colunas sejam numéricas. Sendo assim, as colunas "Sex" e "Embarked" serão categorizadas em números:
* __Sex__
    * *male* passa a ser 0
    * *female* passa a ser 1
* __Embarked__
    * *C* passa a ser 0
    * *Q* passa a ser 1
    * *S* passa a ser 2

In [ ]:
dfTotal['Sex'] = dfTotal['Sex'].replace('male',0)
dfTotal['Sex'] = dfTotal['Sex'].replace('female',1)

dfTotal['Embarked'] = dfTotal['Embarked'].replace('C',0)
dfTotal['Embarked'] = dfTotal['Embarked'].replace('Q',1)
dfTotal['Embarked'] = dfTotal['Embarked'].replace('S',2)

#### Analisando integridade dos dados

É importante saber se os dados estão dispostos de forma íntegra. Uma das análises realizadas no presente trabalho será analisar, caso a caso, se a disposição de categorias para cada coluna está feita da maneira correta. 
* Para a coluna __"Survived"__ os valores possíveis são 0 e 1
* Para a coluna __"Sex"__ os valores possíveis são 0 e 1
* Para a coluna __"Pclass"__ os valores possíveis são 1, 2 e 3
* Para a coluna __"Embarked"__ os valores possíveis são 0, 1 e 2

⚠️ __A coluna *Age* representa a idade de cada tripulante. Desta forma não faz sentido categorizar a idade, muito embora ela seja analisada de forma matemática mais adiante. Da mesma forma, a coluna *Fare* representa o preço e também será analisado de forma matemática mais adiante. Parch e SibSp representam, respectivamente número de pais/filhos tripulantes e número de irmãos/cônjuges a bordo, e, desta forma, também não é interessante tratá-los de forma categórica.__ ⚠️


In [ ]:
print("Analisando valores de Survived:")
dfTotal.groupby(['Survived']).count()

In [ ]:
print("Analisando valores de Sex:")
dfTotal.groupby(['Sex']).count()

In [ ]:
print("Analisando valores de Pclass:")
dfTotal.groupby(['Pclass']).count()

In [ ]:
print("Analisando valores de Embarked:")
dfTotal.groupby(['Embarked']).count()

✅ __As 4 colunas listadas estão categorizadas de forma adequada, não havendo nenhuma linha categorizada com uma classe que não exista.__ ✅

#### Análise de Variância, Desvio Padrão e Outliers

As colunas que não foram analisadas e não estão dispostas em classes devem ser analisadas de outra forma. Sendo assim, cabe uma análise para conhecer variância, desvio padrão e outliers para as colunas:
* Age
* Fare
* SibSp
* Parch

##### Age

In [ ]:
mean  = np.mean(dfTotal['Age'], axis=0)
sd    = np.std(dfTotal['Age'], axis=0)

print("Média:",mean)
print("Desvio padrão:",sd)

In [ ]:
limitMax = mean + 2*sd

print("Limite de idade para outliers:",limitMax)

In [ ]:
print("Gráfico de Outliers")
sns.boxplot(x=dfTotal['Age'])

In [ ]:
print("Idade máxima:",max(dfTotal['Age']))
print("Idade mínima:",min(dfTotal['Age']))

⚠️ __Muito embora o limite de idade calculado (idades maiores que o limite calculado são os outliers) seja pouco mais que 57 anos, observe que a maior idade foi 80. Sendo assim, é notório que estes outliers podem ser considerados na análise do IA, enriquecem a análise e não trazem danos e serão mantidos no database.__ ⚠️

##### Fare

In [ ]:
mean  = np.mean(dfTotal['Fare'], axis=0)
sd    = np.std(dfTotal['Fare'], axis=0)

print("Média:",mean)
print("Desvio padrão:",sd)

In [ ]:
limitMax = mean + 2*sd

print("Limite de tarifa para outliers:",limitMax)

In [ ]:
print("Gráfico de Outliers")
sns.boxplot(x=dfTotal['Fare'])

In [ ]:
print("Tarifa máxima: £",max(dfTotal['Fare']))
print("Tarifa mínima: £",min(dfTotal['Fare']))

❗️ __As análises matemáticas acima para a coluna de tarifa requerem uma observação um pouco mais subjetiva.__ ❗️

Em primeiro lugar observe a Tarifa mínima paga por um tripulante: £0,00. Isso significa que ele entrou de graça. O que isso pode significar. Há a possibilidade de ser apenas um erro no database, como também há a possibilidade de ser algum tipo de "promoção" dedicada a algum critério, ou simplesmente tripulantes que "pularam a catraca". 
Para validar ou invalidar será analisado quantas pessoas entraram sem pagar na tripulação. Com essa informação, caso mais de uma pessoa participe, será entendido que pode ter sido casos de pessoas que puderam entrar sem pagar e, portanto, devem ser consideradas no IA. Caso contrário o usuário será desconsiderado e removido do database.

Da mesma forma, para a tarifa máxima no gráfico é possível observar uma grande diferença da maior tarifa paga em relação às demais e será feita uma análise de quantas pessoas pagaram mais de £500,00.

In [ ]:
print("Quantidade de não pagantes:",len(dfTotal.loc[dfTotal['Fare']==0]))
print("Quantidade de pagantes acima de £500,00:",len(dfTotal.loc[dfTotal['Fare']>=500]))

⚠️ __Observe que tanto para não pagantes quanto para pagantes de um valor acima de £500 não foram casos isolados. Entende-se, então que houve não pagantes na tripulação, bem como houve pessoas com extremas condições para pagar mais de £500,00 pela viagem e, sendo assim, todos os dados referentes a Tarifa serão mantidos.__ ⚠️

##### SibSp

In [ ]:
mean  = np.mean(dfTotal['SibSp'], axis=0)
sd    = np.std(dfTotal['SibSp'], axis=0)

print("Média:",mean)
print("Desvio padrão:",sd)

In [ ]:
limitMax = mean + 2*sd

print("Limite de quantidade de irmãos/cônjuges para outliers:",limitMax)

In [ ]:
print("Gráfico de Outliers")
sns.boxplot(x=dfTotal['SibSp'])

In [ ]:
print("Quantidade de irmãos/cônjuges máxima:",max(dfTotal['SibSp']))
print("Quantidade de irmãos/cônjuges mínima:",min(dfTotal['SibSp']))

✅ __Os dados são completamente satisfatórios, uma vez que é muito possível que uma pessoa contenha 8 irmãos e o cônjuge ou que não tenha nem irmãos nem cônjuge na viagem, não sendo adequado fazer nenhuma remoção.__ ✅

##### Parch

In [ ]:
mean  = np.mean(dfTotal['Parch'], axis=0)
sd    = np.std(dfTotal['Parch'], axis=0)

print("Média:",mean)
print("Desvio padrão:",sd)

In [ ]:
limitMax = mean + 2*sd

print("Limite de quantidade de pais/filhos para outliers:",limitMax)

In [ ]:
print("Gráfico de Outliers")
sns.boxplot(x=dfTotal['Parch'])

In [ ]:
print("Quantidade de pais/filhos máxima:",max(dfTotal['Parch']))
print("Quantidade de pais/filhos mínima:",min(dfTotal['Parch']))

✅ __Os dados são completamente satisfatórios, uma vez que é muito possível que uma pessoa contenha 9 filhos e os pais ou que não tenha nem filhos nem pais na viagem, não sendo adequado fazer nenhuma remoção.__ ✅

#### Analisando a correlação

É importante analisar a correlação das colunas, uma vez que colunas podem apresentar informações redundantes ou quase redundantes, causando uma falta de necessidade de uma delas.

In [ ]:
corr = dfTotal.corr()
corr.style.background_gradient(cmap='coolwarm')

❗️ __Observa-se que há uma correlação entre Sexo e Sobreviventes. Isso pode está associado à quantidade de pessoas e à prioridade de uso dos botes durante o naufragio do navio. Sendo assim a coluna será mantida, uma vez que não há muitas colunas e a correlação possui uma explicação factível, mas ao meu entender, não tão grande a ponto de viabilizar a exclusao da coluna de Sexo.__ ❗️